# Debugging for external process run by Subprocess

In [2]:
from pymeos.db.psycopg import MobilityDB
from pymeos import *
from datetime import datetime, timedelta
import time
from collections import deque
from pympler import asizeof
import gc
from enum import Enum
import numpy as np
from shapely.geometry import Point
import math
import subprocess
import shutil
import os
import sys

DIRECTORY_PATH = os.getcwd()
MATRIX_DIRECTORY_PATH = f'{DIRECTORY_PATH}/matrices'

# AIS Danish maritime dataset
DATABASE_NAME = "mobilitydb"
TPOINT_TABLE_NAME = "processed_data"
TPOINT_ID_COLUMN_NAME = "MMSI"
TPOINT_COLUMN_NAME = "traj"


class Database_connector:
    """
    Singleton class used to connect to the MobilityDB database.
    """
    
    def __init__(self):
        try: 
            connection_params = {
            "host": "localhost",
            "port": 5432,
            "dbname": DATABASE_NAME,
            "user": "postgres",
            "password": "postgres"
            }
            self.table_name = TPOINT_TABLE_NAME
            self.id_column_name = TPOINT_ID_COLUMN_NAME
            self.tpoint_column_name = TPOINT_COLUMN_NAME                  
            self.connection = MobilityDB.connect(**connection_params)

            self.cursor = self.connection.cursor()

            self.cursor.execute(f"SELECT {self.id_column_name} FROM public.{self.table_name};")
            self.ids_list = self.cursor.fetchall()
            self.ids_list = self.ids_list[:int(len(self.ids_list)*PERCENTAGE_OF_OBJECTS)]
        except Exception as e:
            pass

  
    def get_subset_of_tpoints(self, pstart, pend, xmin, ymin, xmax, ymax):
        """
        For each object in the ids_list :
            Fetch the subset of the associated Tpoints between the start and end timestamps
            contained in the STBOX defined by the xmin, ymin, xmax, ymax.
        """
        try:
           
            ids_list = [ f"'{id[0]}'"  for id in self.ids_list]
            ids_str = ', '.join(map(str, ids_list))
          
            query = f"""
                    SELECT 
                        atStbox(
                            a.{self.tpoint_column_name}::tgeompoint,
                            stbox(
                                ST_MakeEnvelope(
                                    {xmin}, {ymin}, -- xmin, ymin
                                    {xmax}, {ymax}, -- xmax, ymax
                                    4326 -- SRID
                                ),
                                tstzspan('[{pstart}, {pend}]')
                            )
                        )
                    FROM public.{self.table_name} as a 
                    WHERE a.{self.id_column_name} in ({ids_str})
                        AND a.{self.tpoint_column_name} IS NOT NULL;
                    """
            self.cursor.execute(query)
            rows = self.cursor.fetchall()
            return rows
        except Exception as e:
            self.log(e)


    def get_min_timestamp(self):
        """
        Returns the min timestamp of the tpoints columns.

        """
        try:
            
            self.cursor.execute(f"SELECT MIN(startTimestamp({self.tpoint_column_name})) AS earliest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass

    def get_max_timestamp(self):
        """
        Returns the max timestamp of the tpoints columns.

        """
        try:
            self.cursor.execute(f"SELECT MAX(endTimestamp({self.tpoint_column_name})) AS latest_timestamp FROM public.{self.table_name};")
            return self.cursor.fetchone()[0]
        except Exception as e:
            pass


    def close(self):
        """
        Close the connection to the MobilityDB database.
        """
        self.cursor.close()
        self.connection.close()




FPS_DEQUEUE_SIZE = 5 # Length of the dequeue to calculate the average FPS
TIME_DELTA_DEQUEUE_SIZE =  10 # Length of the dequeue to keep the keys to keep in the buffer


PERCENTAGE_OF_OBJECTS = 1 # To not overload the memory, we only take a percentage of the ships in the database
TIME_DELTA_SIZE = 240 # Number of frames associated to one Time delta
GRANULARITY = timedelta(minutes=1) # Time delta between two frames
SRID = 4326
FPS = 60



  

In [3]:
db = Database_connector()

start_date = db.get_min_timestamp()
end_date = db.get_max_timestamp()
total_frames = math.ceil( (end_date - start_date) // GRANULARITY )

timestamps = [start_date + i * GRANULARITY for i in range(total_frames)]
timestamps = [dt.replace(tzinfo=None) for dt in timestamps]
timestamps_strings = [dt.strftime('%Y-%m-%d %H:%M:%S') for dt in timestamps]

In [4]:
x_min = -180
y_min = -90
x_max = 180
y_max = 90

start_frame= 0
end_frame = 479

arguments = [start_frame, end_frame, PERCENTAGE_OF_OBJECTS, x_min, y_min, x_max, y_max]
arguments = [str(arg) for arg in arguments]
arguments += [timestamps_strings[0]  , str(len(timestamps)), "MINUTE", MATRIX_DIRECTORY_PATH, DATABASE_NAME, TPOINT_TABLE_NAME, TPOINT_ID_COLUMN_NAME, TPOINT_COLUMN_NAME]
            

In [5]:
arguments = ['0', '59', '1', '-180', '-90', '180', '90', '2023-06-01 00:00:00', '1440', 'SECOND', '/home/ali/matrices/60_1', 'mobilitydb', 'processed_data', 'MMSI', 'traj']
arguments

['0',
 '59',
 '1',
 '-180',
 '-90',
 '180',
 '90',
 '2023-06-01 00:00:00',
 '1440',
 'SECOND',
 '/home/ali/matrices/60_1',
 'mobilitydb',
 'processed_data',
 'MMSI',
 'traj']

In [6]:
# Command to execute Program B
command = ['/usr/bin/python3', '/home/ali/QGIS-MobilityDB/experiment8_subprocess/matrix_file_processed.py', *arguments]
result = subprocess.run(command, capture_output=True, text=True)

In [7]:
result

CompletedProcess(args=['/usr/bin/python3', '/home/ali/QGIS-MobilityDB/experiment8_subprocess/matrix_file_processed.py', '0', '59', '1', '-180', '-90', '180', '90', '2023-06-01 00:00:00', '1440', 'SECOND', '/home/ali/matrices/60_1', 'mobilitydb', 'processed_data', 'MMSI', 'traj'], returncode=0, stdout='================================================================     Matrix 0 created in 0.1654651165008545 seconds, 4.963953495025635 frames for 30 fps animation.\n', stderr='')

In [9]:
loaded_matrix = np.load(f"/home/ali/matrices/matrix_{0}.npy", allow_pickle=True)

In [11]:
np.count_nonzero(loaded_matrix != 'POINT EMPTY')

1595